# Solución de MDPs por medio de iteración de valores

Para este laboratorio utilizaremos el método de iteración de valores para solucionar el problema del motor del vehículo definido en el laboratorio de MDPs.

Para ello utilizaremos el ambiente definido de los motores y construiremos el método de iteración de valores para resolverlo.

## Ambiente

Para recordar la definición del ambiente se realiza en la clase `MDP`. Esta clase recibe como parámetro una codificación de los estados, la dimensión del MDP dada por la cantidad de estados en el ambiente (dados como una tupla) y el estado inicial. 

La codificación del ambiente se define como una tabla de tamaño `estados x acciones` que para cada uno de los estados (filas) contiene una lista de tuplas con la información de la probabilidad asociada a la acción de la columna, el estado de llegada de ejecutar la acción y la recompensa asociada a la ejecución de la acción. La codificación de la tabla se muestra a continuación, donde los estados son `cold=0`, `warm=1` y `overheated=2`.

![Encoding_table](table.png)

Las dimensiones del tablero se almacenan en los astributos `nrows` y `ncols` de la clase `MDP`.
Los valores codificados en el tablero se almacenarán en dos atributos de la clase, un atributo `transitions` que mantiene la probabilidad de transición de un estado a otro dada una acción como una lista de tuplas `(probability, state)` y un atributo `rewards` que mantiene la información las recompensas obtenida en el paso entre estados definido como una lista `(reward, state)`.

Adicionalmente la clase tiene los atributos `initial_state` y `state` que corresponden al estado inicial, dado por parámetro, y el estado actual del agente en el ambiente, inicializado en el estado inicial.

Note también que el estado `overheated` no tienen ninguna acción asociada a él y por lo tanto lo consideramos como un estado final.

In [1]:
import random

class MDP:
    def __init__(self, table, dimensions, initial_state):
        self.nrows, self.ncols = dimensions
        self.transitions = [[0 for _ in range(self.ncols)] for _ in range(self.nrows)]
        self.rewards = [[0 for _ in range(self.ncols)] for _ in range(self.nrows)]
        
        for i in range(self.nrows):
            for j in range(self.ncols):
                state = table[i][j]
                if state is not None:
                    list_transitions = []
                    list_rewards = []
                    for inf in state:
                        p, s, r = inf
                        list_transitions.append((p,s))
                        list_rewards.append((r,s))
                    self.transitions[i][j] = list_transitions
                    self.rewards[i][j] = list_rewards
                else:
                    self.transitions[i][j] = None
                    self.rewards[i][j] = None
        self.initial_state = initial_state
        self.state = self.initial_state
    
    def get_current_state(self):
        return self.state
    
    def get_possible_actions(self, state):
        actions = ['slow', 'fast']
        if state == 2:
            actions = []
        return actions
    
    def get_action_index(self, action):
        actions = ['slow', 'fast']
        index = 0
        for a in actions:
            if action == a:
                return index
            index += 1
    
    def get_possible_states(self, state, action):
        action_index = self.get_action_index(action)
        new_state = None
        rewards=[]
        states=[]
        probabilities = []
        for i in range(len(self.transitions[state][action_index])):
            prob, new_state = self.transitions[state][action_index][i]
            reward, _ = self.rewards[state][action_index][i]
            probabilities.append(prob)
            rewards.append(reward)
            states.append(new_state)
        return probabilities, rewards, states
        
    def simulate_action(self, state, action):
        act = -1
        if action == 'slow':
            act = 0
        else:
            act = 1
        start_state_action = self.transitions[state][act]
        prob = 0
        index = -1
        new_state = -1
        for op in start_state_action:
            r = random.random() 
            if prob < r and r <= prob + op[0]:
                new_state = op[1]
            prob += op[0]
            index += 1
        return self.rewards[state][act][index][0], new_state
    
    def do_action(self, action):
        act = -1
        state = self.state
        if action == 'slow':
            act = 0
        else:
            act = 1
        start_state_action = self.transitions[state][act]
        prob = 0
        index = -1
        for op in start_state_action:
            r = random.random() 
            if prob < r and r <= prob + op[0]:
                self.state = op[1]
            prob += op[0]
            index += 1
        return self.rewards[state][act][index][0], self.state

    def reset(self):
        self.state = self.initial_state
    
    def is_terminal(self, state):
        return self.get_possible_actions(state) == []

## Creación del agente 

#### 1. definición de la clase de iteración de valores

Implemente la clase `ValueIteration` que define cuatro atributos:

- `mdp` que corresponde al MDP a resolver.
- `discount` que corresponde al factor de decuento a utilizar.
- `iterations` que corresponde a el número de iteraciones a realizar
- `values` que corresponde a un mapa con los valores calculados para los estados del MDP. Los estados se definen como una tupla de los valores posibles del estado. Como simil, en el caso de Gridworld la tupla es la posición de cada casilla. Inicialmente definiremos el mapa como un mapa vacío, el cual poblaremos con los estados descubiertos durante cada iteración.

Al momento de crear la clase `ValueIteration`, esta debe recibir por parámetro: la definición del MDP `MDP`, el valor de descuento `discount` (inicializado por defecto en 0.9 si no se pasa ningún valor) y la cantidad de iteraciones a ejecutar `iterations` (con un valor de 30 por defecto, si no se pasa ).

In [2]:
class ValueIteration():    
    def __init__(self, mdp, discount = 0.9, iterations = 30):
        self.mdp = mdp
        self.discount = discount
        self.iterations = iterations
        self.values = {}
    
    def get_value(self, state):
        return self.values.get(state, 0)

    def compute_new_value(self, state):
        if self.mdp.is_terminal(state):
            return 0 
        
        actions = self.mdp.get_possible_actions(state)
        if not actions:
            return 0
        
        action_values = []
        for a in actions:
            prob, rewards, next_states = self.mdp.get_possible_states(state, a)
            value_sum = 0
            for p, r, s_prime in zip(prob, rewards, next_states):
                value_sum += p * (r + self.discount * self.get_value(s_prime))
            action_values.append(value_sum)
        return max(action_values) if action_values else 0


    def get_action(self, state):
        if self.mdp.is_terminal(state):
            return None
        actions = self.mdp.get_possible_actions(state)
        if not actions:
            return None

        best_action = None
        best_value = float('-inf')
        for a in actions:
            prob, rewards, next_states = self.mdp.get_possible_states(state, a)
            value_sum = 0
            for p, r, s_prime in zip(prob, rewards, next_states):
                value_sum += p * (self.discount * self.get_value(s_prime))
            if value_sum > best_value:
                best_value = value_sum
                best_action = a
        return best_action
        
    def run_value_iteration(self):
        for _ in range(self.iterations): 
            new_values = {}
            for state in range(self.mdp.nrows):
                new_values[state] = self.compute_new_value(state)
            self.values = new_values

class ValueIteration(ValueIteration):
    def get_policy(self) -> dict:
            self.run_value_iteration()
            policy = {}
            for state in range(self.mdp.nrows):
                policy[state] = self.get_action(state)
            return policy

In [3]:
# Definición del ambiente y creación del MDP
board = [[[(1,0,1)], [(0.5, 0, 2), (0.5,1,2)]],
         [[(0.5, 0, 1), (0.5,1,1)], [(1, 2, -10)]],
         [None, None]]

env = MDP(board, (3,2), 0)

In [4]:
#Pruebas estructura del agente
a = ValueIteration(env, 0.1, 10)

try:
    a.mdp
    assert type(a.mdp) is MDP, "El tipo del mdp debe ser MDP (el tipo de la clase)"
except:
    print("El atributo mdp no está definido")
try:
    a.discount
    assert type(a.discount) is float, "El tipo del factor de descuento debe ser float"
    assert 0 < a.discount and a.discount <=1, "El factor de descuento debe ser un valor entre 0 (excluido) y 1 (incluido)"
except:
    print("El atributo discount no está definido")
try:
    a.iterations
    assert type(a.iterations) is int, "El tipo de la cantidad de iteraciones debe ser entero"
except:
    print("El atributo iterations no está definido")

try:
    a.values
    assert type(a.values) is dict or type(a.values) is dict[int,float], "El tipo del mapa de valeres debe ser dict (el tipo de los mapas en python). El mapa puede estar isntanciado en su llave y valor si fue inicializado previamente"
except:
    print("El atributo values no está definido")
    
    

In [5]:
#Pruebas adicionales


#### 2. Valores de estados 

Defina la función `get_value` que recibe un estado (el entero representando el estado) como parámetro y retorna el valor correspondiente para dicho estado. Si el estado no ha sido visitado, la función debe retornar 0, el valor inicial para el estado.

In [6]:
#Pruebas de get_value

a = ValueIteration(env, 0.8, 10)

try:
    a.get_value
except:
    print("La función get_value no está definida")

assert type(a.get_value(0)) is int or type(a.get_value(0)) is float, "La función debe retornar un valor entero o un flotante."


In [7]:
#Pruebas adicionales


#### 3. Cálculo de valor 

Defina la función `compute_new_value` que recibe un estado (entero en este caso) y calcula el nuevo valor para el estado siguiendo la fórmula de iteración de valores. Esta función calcula el nuevo valor para el estado actual del agente (`s`) a partir de las acciones, las recompensas y los valores de los posibles estados de llegada (`s'`) de acuerdo a la función de transición (o ruido). El nuevo valor calculado debe ser el valor máximo de todas las posibles acciones en el estado de entrada.

Notas:
- El valor inicial de todos los estados debe ser 0.
- Para poder realizar este cálculo requerimos la nueva función del ambiente `get_possible_states`. La nueva función recibe el nombre de la acción a ejecutar para el estado actual del ambiente y retorna una lista con las probabilidades de transiciones entre los estados, una lista de recompensas y una lista de los estados de llegada para la acción desde el estado actual.

In [8]:
#Pruebas de compute_new_value

a = ValueIteration(env, 0.8, 10)

try:
    a.compute_new_value
except:
    print("La función compute_new_value no está definida")

assert type(a.compute_new_value(0)) is int or type(a.compute_new_value(0)) is float, "La función compute_new_value debe retornar un valor de tipo entero o flotante"


In [9]:
#Pruebas adicionales


#### 4. Acción a ejecutar 

Defina la función `get_action` que retorna la acción a tomar para un estado dado por parámetro, de acuerdo a los valores de los estados aledaños. Esta función debe observar el posible valor a obtener para cada una de sus acciones posibles y retornar aquella acción (su nombre) que lleva al mejor valor.
Tenga en cuenta que la evaluación de las acciones no debe modificar el valor actual del estado, o tener un efecto visible sobre el ambiente o el agente.

In [10]:
#Pruebas de get_action

a = ValueIteration(env, 0.8, 50)

try:
    a.get_action
except:
    print("La función get_action no está definida")

assert type(a.get_action(0)) is str, "La función get_action debe retornar el nombre de una acción tipo string"

In [11]:
#Pruebas adicionales


#### Preguntas Adicionales

Calcule la política óptima para el agente. Para cumplir esta tarea debe definir una función, que ejecute la iteración de valores y retorne un diccionario donde para cada estado se retorna la acción óptima a ejecutar.

¿Los valores convergen, porqué o porqué no?
¿Hay alguna diferencia si comienza desde el estado frio, caliente, o sobrecalentado?

In [12]:
print('ok')

ok
